In [161]:
import pyodbc
import datetime
from datetime import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


ip_date = "25-02-2022"
day, month, year = map(int, ip_date.split('-'))
ip_date = datetime.datetime(year, month, day)
ip_start_time = "08:32:00"
h, m, s = map(int, ip_start_time.split(':'))
ip_start_time = datetime.datetime(year, month, day, h, m, 00)
print(ip_start_time)
ip_start_end_time = datetime.datetime(year, month, day, h, m, 59)
print(ip_start_end_time)
ip_end_time = "09:04:00"
# eh, em, es = map(int, ip_end_time.split(':'))
# ip_end_time = time(hour=eh, minute=em, second=es)
# ip_start_dt = ip_date.combine(ip_date, ip_start_time)
# ip_end_dt = ip_date.combine(ip_date, ip_end_time)
dvalue_code1 = 0.92
dvalue_code2 = 0.97
dvalue_code3 = 0.90
dvalue_code4 = 0.81
dvalue_code5 = 0.8
dvalue_code6 = 0.95
d_values = [dvalue_code1, dvalue_code2, dvalue_code3,
            dvalue_code4, dvalue_code5, dvalue_code6]
tolerance = 0.01

2022-02-25 08:32:00
2022-02-25 08:32:59


In [162]:
connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=GOVIND\MSSQLSERVER01;DATABASE=backtest;Trusted_Connection=yes;')
query = """select * from dbo.Day3;"""
df = pd.read_sql(query, connection)


In [163]:
df['start_date_time'] = df.apply(lambda r : pd.datetime.combine(r['Date'],r['Time']),1)
df = df.drop(['Date', 'Time'], axis = 1)
df = df.dropna()
df.head()

In [176]:
df = df[df.start_date_time.between(ip_start_time, ip_start_end_time)]
unique_final_dates = df['FinalDate'].unique()
final_date_dict = {}

for final_date in unique_final_dates:
    final_date_dict[final_date] = {"data":df[df['FinalDate'] == final_date]}
    print(final_date_dict[final_date]["data"].head())

           Code  Dvalue  Price   FinalDate     start_date_time
1023  Code4310P   -0.56  47.70  2022-02-28 2022-02-25 08:32:00
1024  Code4315P   -0.58  50.25  2022-02-28 2022-02-25 08:32:00
1025  Code4320P   -0.60  53.05  2022-02-28 2022-02-25 08:32:00
1026  Code4325P   -0.62  55.90  2022-02-28 2022-02-25 08:32:01
1027  Code4330P   -0.64  59.45  2022-02-28 2022-02-25 08:32:01
           Code  Dvalue   Price   FinalDate     start_date_time
1062  Code4100C     1.0  194.90  2022-02-25 2022-02-25 08:32:06
1063  Code4105C     1.0  190.00  2022-02-25 2022-02-25 08:32:07
1064  Code4110C     1.0  185.00  2022-02-25 2022-02-25 08:32:07
1065  Code4115C     1.0  180.10  2022-02-25 2022-02-25 08:32:07
1066  Code4120C     1.0  175.15  2022-02-25 2022-02-25 08:32:07


In [184]:
for key_date in final_date_dict.keys():
    temp = final_date_dict[key_date]['data']
    df_dval_filtered = pd.DataFrame([], columns=list(temp.columns))
    type(df_dval_filtered)
    avail_dvals = list(temp['Dvalue'].unique())
    dvalue_first = pd.DataFrame([], columns=list(temp.columns))
    for d_value in d_values:
        curr_dvalue = d_value
        df_dvals = temp[temp['Dvalue'] == curr_dvalue]
        while len(df_dvals) == 0:
            # print(len(df_dvals), d_value)
            less_curr_dvalue, high_curr_dvalue = curr_dvalue - tolerance, curr_dvalue + tolerance
            if less_curr_dvalue in avail_dvals:
                curr_dvalue = less_curr_dvalue
                df_dvals = temp[temp['Dvalue'] == curr_dvalue]
            elif high_curr_dvalue in avail_dvals:
                curr_dvalue = high_curr_dvalue
                df_dvals = temp[temp['Dvalue'] == curr_dvalue]
        #print(df_dvals.iloc[0])
        dvalue_first = pd.concat([dvalue_first, df_dvals.head(1)], axis=0, ignore_index=True)
        df_dval_filtered = pd.concat([df_dval_filtered, df_dvals], axis=0, ignore_index=True)
        # final_date_dict_filtered[]
        #final_date_dict[key_date]["filtered"] = df_dval_filtered
        final_date_dict[key_date]["first"] = dvalue_first
    
    # df_dval_filtered

In [186]:
df.tail(1)

,Code,Dvalue,Price,FinalDate,start_date_time
2443,Code4330P,-0.62,54.95,2022-02-28,2022-02-25 08:32:59


In [185]:
for val in final_date_dict.values():
    print(val["first"])


        Code Dvalue   Price   FinalDate      start_date_time
0  Code4115C   0.92  184.35  2022-02-28  2022-02-25 08:32:15
1  Code4080C   0.97  221.65  2022-02-28  2022-02-25 08:32:55
2  Code4135C    0.9  165.45  2022-02-28  2022-02-25 08:32:15
3  Code4195C   0.81   111.2  2022-02-28  2022-02-25 08:32:16
4  Code4200C    0.8  107.95  2022-02-28  2022-02-25 08:32:16
5  Code4075C   0.95   222.8  2022-02-28  2022-02-25 08:32:15
        Code Dvalue   Price   FinalDate      start_date_time
0  Code4130C   0.92   171.7  2022-02-25  2022-02-25 08:32:37
1  Code4100C   0.97  200.65  2022-02-25  2022-02-25 08:32:37
2  Code4175C    0.9   122.3  2022-02-25  2022-02-25 08:32:07
3  Code4215C   0.81    89.5  2022-02-25  2022-02-25 08:32:38
4  Code4220C    0.8    80.4  2022-02-25  2022-02-25 08:32:08
5  Code4120C   0.95  180.85  2022-02-25  2022-02-25 08:32:37
